## RF with 3 categories: No matter, NS but no Remnant, NS and a remnant
## FINAL TRAINING IN O2 DATA. PARAMETER TUNING ON 90-10

In [1]:
import classyRF_final as classyRF
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
import importlib
importlib.reload(classyRF)

<module 'classyRF_final' from 'C:\\Users\\marin\\Desktop\\ML_workingGroup\\FINAL\\classyRF_final.py'>

In [10]:
pathREM = "C:\\Users\\marin\\Desktop\\ML_workingGroup\\RM_dataset_NOPUBLIC\\"
pathBNS = "C:\\Users\\marin\\Desktop\\ML_workingGroup\\NS_dataset_NOPUBLIC\\"
pathClassy = "C:\\Users\\marin\\Desktop\\ML_workingGroup\\classy_RF\\"
pathData = "C:\\Users\\marin\\Desktop\\ML_workingGroup\\allEOS\\"
pathTrees = "C:\\Users\\marin\\Desktop\\ML_workingGroup\\FINAL\\Forest_all_O2\\"
pathMDC = "C:\\Users\\marin\\Desktop\\ML_workingGroup\\FINAL\\"

In [4]:
i_m1r=1; i_m2r=2; i_x1r=3; i_x2r=4; i_mcr=5; i_qr=6; i_Rr=7;
i_snr=8;
recovered_index = [i_m1r, i_m2r, i_x1r, i_x2r, i_mcr, i_qr, i_Rr, i_snr]
independent_recovered_index = [i_m1r, i_m2r, i_x1r, i_x2r, i_snr]

# Do it all loop

In [5]:
EOS = ["APR4_EPP", "BHF_BBB2", "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1_PP", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLy", "SLY2", "SLY9", "SLY230A"]

random.seed(42)
plt.rcParams["font.size"]=14

In [7]:
from sklearn.ensemble import RandomForestClassifier
import time
def hyperparam(obj,trees=[10,30,50,80,100], info=['entropy', 'gini'], features=[5,'sqrt'],depths=[10,15,20,25]):
    best_score = -1
    scores=[]

    for tree in trees:
        for criteria in info:
            for feature in features:
                for depth in depths:
                    if obj.verbose:
                        print("Doing ",tree, " trees, criterion ",criteria," and ",feature," features. Depth: ", depths) 
                    clf = RandomForestClassifier(n_estimators=tree, criterion=criteria, max_features=feature, max_depth=depth,random_state=42) 
                    t0=time.perf_counter()
                    clf.fit(obj.data_train, np.ravel(obj.labels_train))
                    total_time=time.perf_counter()-t0
                    score = clf.score(obj.data_test,obj.labels_test)
                    print("Training: ",tree, " trees, criterion ",criteria," and ",feature," features. Depth ",depth,"Score: ",score,". Time elapsed: {:.3f}".format(total_time), "s.")
                    scores.append(score)
                    if score>best_score:
                        best_score = score
                        config=[tree,criteria,feature,depth]

    print("Standard deviation of score during crossvalidation: ",np.std(scores),". Mean: ",np.mean(scores))
    
    print("*********************************************************")
    print("Trees       Criteria       Features     Depth     SCORE")
    k=0
    for tree in trees:
        for criteria in info:
            for feature in features:
                for depth in depths:
                    print(tree,"   ", criteria,"   ", feature, "   ", depth, "   ", scores[k])
                    k=k+1
        print("--------------------------------------------------------")
    print("***********************************************************")
    

    print("Higher score: ",best_score,". Optimum forest found: ",config[0]," trees, ",config[1], " criteria and ",config[2]," max features", config[3], " depth")
    return config

In [11]:
score_list=[]

for eos in EOS:
    print("----------------------------------------------------------")
    print("Doing", eos)
    print("----------------------------------------------------------")
    RF = classyRF.ClassificationRF()
    RF.load_original_dataset(pathData+eos+'\\EMB\\', fname_x='original_data_'+eos+'_s300_f0d7.csv')
    
    N=int(len(RF.data_train_all));print(N, "total events")
    test_N=int(np.floor(0.1*N)); print(test_N, "events for testing")
    indexes_test=list(np.arange(0,test_N));
    all_indexes=(np.arange(0,N)).tolist()
    indexes_train=list(set(all_indexes) - set(indexes_test))

    labelREM=RF.data_train_all[indexes_train,-1]
    labelBNS=RF.data_train_all[indexes_train,-2]
    RF.labels_train = labelREM+labelBNS

    RF.data_test_all=RF.data_train_all[indexes_test,0:-2]
    labelREM=RF.data_train_all[indexes_test,-1]
    labelBNS=RF.data_train_all[indexes_test,-2]
    RF.labels_test = labelREM+labelBNS

    RF.data_train_all=RF.data_train_all[indexes_train,0:-2]
    
    RF.subset_features(independent_recovered_index)
    
    config=hyperparam(RF)




----------------------------------------------------------
Doing APR4_EPP
----------------------------------------------------------
loaded
198961 total events
19896 events for testing
Training:  10  trees, criterion  entropy  and  5  features. Depth  10 Score:  0.9707981503819864 . Time elapsed: 8.354 s.
Training:  10  trees, criterion  entropy  and  5  features. Depth  15 Score:  0.9720044229995979 . Time elapsed: 9.530 s.
Training:  10  trees, criterion  entropy  and  5  features. Depth  20 Score:  0.9720044229995979 . Time elapsed: 9.919 s.
Training:  10  trees, criterion  entropy  and  5  features. Depth  25 Score:  0.9704463208685162 . Time elapsed: 9.991 s.
Training:  10  trees, criterion  entropy  and  sqrt  features. Depth  10 Score:  0.9695918777643747 . Time elapsed: 3.599 s.
Training:  10  trees, criterion  entropy  and  sqrt  features. Depth  15 Score:  0.9718536389223965 . Time elapsed: 4.340 s.
Training:  10  trees, criterion  entropy  and  sqrt  features. Depth  20 Scor

Training:  100  trees, criterion  entropy  and  5  features. Depth  20 Score:  0.972708082026538 . Time elapsed: 98.252 s.
Training:  100  trees, criterion  entropy  and  5  features. Depth  25 Score:  0.9721049457177322 . Time elapsed: 99.185 s.
Training:  100  trees, criterion  entropy  and  sqrt  features. Depth  10 Score:  0.9702955367913149 . Time elapsed: 36.290 s.
Training:  100  trees, criterion  entropy  and  sqrt  features. Depth  15 Score:  0.9728588661037394 . Time elapsed: 43.485 s.
Training:  100  trees, criterion  entropy  and  sqrt  features. Depth  20 Score:  0.9726075593084037 . Time elapsed: 45.521 s.
Training:  100  trees, criterion  entropy  and  sqrt  features. Depth  25 Score:  0.971702854845195 . Time elapsed: 46.376 s.
Training:  100  trees, criterion  gini  and  5  features. Depth  10 Score:  0.9707478890229192 . Time elapsed: 64.332 s.
Training:  100  trees, criterion  gini  and  5  features. Depth  15 Score:  0.9720044229995979 . Time elapsed: 83.826 s.
Trai

NameError: name 'tress' is not defined

The better ones, although very similar, are:
Training:  30  trees, criterion  entropy  and  5  features. Depth  15 Score:  0.9727583433856052 . Time elapsed: 28.543 s
Training:  30  trees, criterion  entropy  and  sqrt  features. Depth  20 Score:  0.9728086047446723 . Time elapsed: 13.814 s.
Training:  50  trees, criterion  entropy  and  5  features. Depth  15 Score:  0.9730096501809409 . Time elapsed: 48.296 s.
Training:  80  trees, criterion  entropy  and  5  features. Depth  15 Score:  0.9733112183353438 . Time elapsed: 79.403 s.

It seems clear than 5 features give best results, also a maximum depth around 15. The entropy information criteria is common to the larger scores.

Let's fix trees=[30,50,80] with entropy criteria, 5 features and 15 depth. Let's crossvalidate for all EOS



In [14]:
def hyperparam_refined(eos,obj,trees=[30,50,80], info=['entropy', 'gini'], features=[5,'sqrt'],depths=[10,15,20,25]):
    best_score = -1
    scores=[]

    for tree in trees:
        clf = RandomForestClassifier(n_estimators=tree, criterion='entropy', max_features=5, max_depth=15,random_state=42) 
        t0=time.perf_counter()
        clf.fit(obj.data_train, np.ravel(obj.labels_train))
        total_time=time.perf_counter()-t0
        score = clf.score(obj.data_test,obj.labels_test)
        scores.append(score)
        if score>best_score:
            best_score = score
            config=[tree]

    print("Standard deviation of score during crossvalidation: ",np.std(scores),". Mean: ",np.mean(scores))
    
    print("*********************************************************")
    print("EOS    Trees       Criteria       Features     Depth     SCORE")
    k=0
    for tree in trees:
        print(eos, "   ", tree,"'entropy'      5     15  ", scores[k])
        k=k+1
    

    print(eos,": Higher score: ",best_score,". Optimum forest found: ",config[0]," trees")
    return config

In [15]:
score_list=[]

for eos in EOS:
    RF = classyRF.ClassificationRF()
    RF.load_original_dataset(pathData+eos+'\\EMB\\', fname_x='original_data_'+eos+'_s300_f0d7.csv')
    
    N=int(len(RF.data_train_all));print(N, "total events")
    test_N=int(np.floor(0.1*N)); print(test_N, "events for testing")
    indexes_test=list(np.arange(0,test_N));
    all_indexes=(np.arange(0,N)).tolist()
    indexes_train=list(set(all_indexes) - set(indexes_test))

    labelREM=RF.data_train_all[indexes_train,-1]
    labelBNS=RF.data_train_all[indexes_train,-2]
    RF.labels_train = labelREM+labelBNS

    RF.data_test_all=RF.data_train_all[indexes_test,0:-2]
    labelREM=RF.data_train_all[indexes_test,-1]
    labelBNS=RF.data_train_all[indexes_test,-2]
    RF.labels_test = labelREM+labelBNS

    RF.data_train_all=RF.data_train_all[indexes_train,0:-2]
    
    RF.subset_features(independent_recovered_index)
    
    config=hyperparam_refined(eos,RF)




loaded
198961 total events
19896 events for testing
Standard deviation of score during crossvalidation:  0.00022602093491543587 . Mean:  0.9730264039672966
*********************************************************
EOS    Trees       Criteria       Features     Depth     SCORE
APR4_EPP     30 'entropy'      5     15   0.9727583433856052
APR4_EPP     50 'entropy'      5     15   0.9730096501809409
APR4_EPP     80 'entropy'      5     15   0.9733112183353438
APR4_EPP : Higher score:  0.9733112183353438 . Optimum forest found:  80  trees
loaded
193624 total events
19362 events for testing
Standard deviation of score during crossvalidation:  0.00011157147502680648 . Mean:  0.9733498605515959
*********************************************************
EOS    Trees       Criteria       Features     Depth     SCORE
BHF_BBB2     30 'entropy'      5     15   0.9731949178803843
BHF_BBB2     50 'entropy'      5     15   0.9734015081086664
BHF_BBB2     80 'entropy'      5     15   0.973453155665737
B

loaded
199821 total events
19982 events for testing
Standard deviation of score during crossvalidation:  0.0002463020561878234 . Mean:  0.9702565642411504
*********************************************************
EOS    Trees       Criteria       Features     Depth     SCORE
SKI6     30 'entropy'      5     15   0.9702232008807927
SKI6     50 'entropy'      5     15   0.9699729756781102
SKI6     80 'entropy'      5     15   0.9705735161645481
SKI6 : Higher score:  0.9705735161645481 . Optimum forest found:  80  trees
loaded
197566 total events
19756 events for testing
Standard deviation of score during crossvalidation:  0.00011930667159113478 . Mean:  0.969561989606533
*********************************************************
EOS    Trees       Criteria       Features     Depth     SCORE
SKMP     30 'entropy'      5     15   0.9697307147195788
SKMP     50 'entropy'      5     15   0.9694776270500102
SKMP     80 'entropy'      5     15   0.9694776270500102
SKMP : Higher score:  0.969730

APR4_EPP - 80 trees (0.003 better than 50)
BHF_BB2 - 80 trees (0.00005 better than 50)
H4 - 50 trees
HQC18 - 50 trees
KDE0V - 80 trees (0.0003 better than 50)
KDE0V1 - 30 trees (0.0004 better than 50)
MPA1 - 50 trees
MS1_PP - 80 trees (0.0001 better than 50)
MS1B_PP - 30 trees (0.0001 better than 50)
RS - 50 trees
SK255 - 80 trees (0.0004 better than 50)
30
50
50
50
80
80
30
80 skop
30
80
50
50

50 trees look fine. Along entropy criteria, 5 features and 15 depth


In [20]:
score_list=[]

for eos in EOS:
    print("-------------------------------")
    print("Doing", eos)
    RF = classyRF.ClassificationRF()
    RF.load_original_dataset(pathData+eos+'\\EMB\\', fname_x='original_data_'+eos+'_s300_f0d7.csv')
    
    N=int(len(RF.data_train_all));print(N, "total events")
    test_N=int(np.floor(0.0*N)); print(test_N, "events for testing")
    indexes_test=list(np.arange(0,test_N));
    all_indexes=(np.arange(0,N)).tolist()
    indexes_train=list(set(all_indexes) - set(indexes_test))

    labelREM=RF.data_train_all[indexes_train,-1]
    labelBNS=RF.data_train_all[indexes_train,-2]
    RF.labels_train = labelREM+labelBNS

    RF.data_test_all=RF.data_train_all[indexes_test,0:-2]
    labelREM=RF.data_train_all[indexes_test,-1]
    labelBNS=RF.data_train_all[indexes_test,-2]
    RF.labels_test = labelREM+labelBNS

    RF.data_train_all=RF.data_train_all[indexes_train,0:-2]
    
    RF.subset_features(independent_recovered_index)
    
    RF.train(trees=50, criterion='entropy', features=5, max_depth=15)
    
    RF.saveModel(pathTrees, eos)




-------------------------------
Doing APR4_EPP
loaded
198961 total events
0 events for testing
-------------------------------
Doing BHF_BBB2
loaded
193624 total events
0 events for testing
-------------------------------
Doing H4
loaded
195764 total events
0 events for testing
-------------------------------
Doing HQC18
loaded
196070 total events
0 events for testing
-------------------------------
Doing KDE0V
loaded
194264 total events
0 events for testing
-------------------------------
Doing KDE0V1
loaded
194434 total events
0 events for testing
-------------------------------
Doing MPA1
loaded
203407 total events
0 events for testing
-------------------------------
Doing MS1_PP
loaded
203407 total events
0 events for testing
-------------------------------
Doing MS1B_PP
loaded
203407 total events
0 events for testing
-------------------------------
Doing RS
loaded
197789 total events
0 events for testing
-------------------------------
Doing SK255
loaded
198532 total events
0 even

# Load trained model

In [6]:
BayesFactor = [1.526, 1.555, 0.056, 1.422, 1.177, 1.283, 0.276, 0.001, 0.009, 0.176, 0.179, 0.159, 0.108, 0.107, 0.33, 0.025, 0.288, 0.29, 0.618, 1.0, 1.028, 0.37, 0.932]

In [9]:
i_m1r=1; i_m2r=2; i_x1r=3; i_x2r=4; i_snr=8;
recovered=[i_m1r, i_m2r, i_x1r, i_x2r, i_snr]
EOSdicRF = {}
count = 0
for eos in EOS:
    RF = classyRF.ClassificationRF()
    RF.loadModel(pathTrees,eos)
    new_dic = {}
    new_dic['forest'] = RF
    new_dic['weight'] = BayesFactor[count]
    EOSdicRF[eos] = new_dic
    count = count + 1 

loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\APR4_EPP.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\BHF_BBB2.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\H4.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\HQC18.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\KDE0V.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\KDE0V1.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\MPA1.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\MS1_PP.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\MS1B_PP.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\RS.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\SK255.joblib
loading  C:\Users\marin\Desktop\ML_workingGroup\FINAL\Forest_all_O2\SK272.joblib
loading  C:\Users\marin\D

## MDC11

mass1,mass2,spin1z,spin2z,combined_snr,combined_far,pipeline,NS_SLy,EMB_SLy,NS_BHF_BBB2,EMB_BHF_BBB2,NS_KDE0V,EMB_KDE0V,NS_KDE0V1,EMB_KDE0V1,NS_SKOP,EMB_SKOP,NS_H4,EMB_H4,NS_HQC18,EMB_HQC18,NS_SLY2,EMB_SLY2,NS_SLY230A,EMB_SLY230A,NS_SKMP,EMB_SKMP,NS_RS,EMB_RS,NS_SK255,EMB_SK255,NS_SLY9,EMB_SLY9,NS_APR4_EPP,EMB_APR4_EPP,NS_SKI2,EMB_SKI2,NS_SKI4,EMB_SKI4,NS_SKI6,EMB_SKI6,NS_SK272,EMB_SK272,NS_SKI3,EMB_SKI3,NS_SKI5,EMB_SKI5,NS_MPA1,EMB_MPA1,NS_MS1B_PP,EMB_MS1B_PP,NS_MS1_PP,EMB_MS1_PP

In [12]:
import pandas as pd
df = pd.read_csv(pathMDC+'MDC11.csv')
df.head()

,mass1,mass2,spin1z,spin2z,combined_snr,combined_far,pipeline,NS_SLy,EMB_SLy,NS_BHF_BBB2,...,NS_SKI3,EMB_SKI3,NS_SKI5,EMB_SKI5,NS_MPA1,EMB_MPA1,NS_MS1B_PP,EMB_MS1B_PP,NS_MS1_PP,EMB_MS1_PP
0,4.268033,3.634232,-0.913736,0.450203,18.991429,7.532970e-13,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
1,6.268945,2.588275,-0.146668,0.099328,19.285233,1.467042e-07,pycbc,1,0,1,...,1,0,1,0,1,0,1,0,1,0
2,12.247071,1.550140,0.370892,0.019994,19.721900,1.387861e-17,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,6.154887,2.612731,-0.016768,-0.532756,18.754279,1.323210e-11,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
4,21.466118,1.100672,0.726591,0.007167,15.509615,4.250779e-06,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0


In [15]:
#Miquel's code: compute number of trees saying each category for every event
def matrix_forest_O3(RFdic, ntrees, events, events_id = None, o3 = False, norm = False ):
    
    total_events = len(events[:,0])
    
    predictions_trees =np.zeros(shape=(ntrees,len(events[:,0])))
    
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    nplus_no = np.zeros(total_events)
    
    for i in range(0,ntrees):
        predictions_trees[i] = RFdic['forest'].model.estimators_[i].predict(events[:])
    
    for i in range(0,total_events):
        for ll in range(0,len(predictions_trees[:,i])): 
            predictions_trees[ll,i] = int(predictions_trees[ll,i])
        labels = list(predictions_trees[:,i])
        if norm == False:
            nplus_NS[i] = int(labels.count(1))
            nplus_REM[i] = int(labels.count(2))
            nplus_no[i] = int(labels.count(0))
        else:    
            nplus_NS[i] = int(labels.count(1))/ntrees
            nplus_REM[i] = int(labels.count(2))/ntrees
            nplus_no[i] = int(labels.count(0))/ntrees
        
        if o3 == True:
            print('Nothing: ',nplus_no[i])
            print('NS: ',nplus_NS[i])
            print('REM: ',nplus_REM[i])
            print('*'*60)
        
    return nplus_NS, nplus_REM, nplus_no

In [26]:
columns = df[['mass1', 'mass2', 'spin1z', 'spin2z', 'combined_snr']]

# Convert the DataFrame to a Numpy array
events = columns.values

for eos in EOS:
    EOSdicRF[eos]['NS_label']= df['NS_'+eos].values
    EOSdicRF[eos]['REM_label']= df['EMB_'+eos].values

In [40]:
EOS = ["APR4_EPP", "BHF_BBB2", "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1_PP", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLy", "SLY2", "SLY9", "SLY230A"]

In [41]:
for eos in EOS:
    nplus_NS = []
    nplus_REM = []
    nplus_NO = []
    print('Doing EOS: ', eos)
    print('='*60)
    npns,nprem,npno = matrix_forest_O3(EOSdicRF[eos], 50, events, o3 = False)
    nplus_NS.append(npns)
    nplus_REM.append(nprem)
    nplus_NO.append(npno)
    nplus_NO = nplus_NO[0]
    nplus_NS = nplus_NS[0]
    nplus_REM = nplus_REM[0]

    with open('C:\\Users\\marin\\Desktop\\ML_workingGroup\\FINAL\\matrices_MDC\\matrix_MDC11_'+eos+'.csv','w') as f:

        f.write('# trueNS,trueEMB,f_0,f_1,f_2 \n')

        for i in range(0,len(events)):
            f.write('%d,%d,%d,%d,%d\n'%(EOSdicRF[eos]['NS_label'][i],EOSdicRF[eos]['REM_label'][i],
                                        nplus_NO[i] ,nplus_NS[i], nplus_REM[i]))

Doing EOS:  APR4_EPP
Doing EOS:  BHF_BBB2
Doing EOS:  H4
Doing EOS:  HQC18
Doing EOS:  KDE0V
Doing EOS:  KDE0V1
Doing EOS:  MPA1
Doing EOS:  MS1_PP
Doing EOS:  MS1B_PP
Doing EOS:  RS
Doing EOS:  SK255
Doing EOS:  SK272
Doing EOS:  SKI2
Doing EOS:  SKI3
Doing EOS:  SKI4
Doing EOS:  SKI5
Doing EOS:  SKI6
Doing EOS:  SKMP
Doing EOS:  SKOP
Doing EOS:  SLy
Doing EOS:  SLY2
Doing EOS:  SLY9
Doing EOS:  SLY230A
